In [1]:
import os
import csv
import sys
import scipy.optimize as opt
import scipy.stats as stat
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la

In [2]:
modulef = 'C:/Users/koolk/Desktop/brain-diffusion/brain_diffusion/Chad/'
sys.path.insert(0, modulef)
from msd import fillin2, MSD_iteration

In [ ]:
df = np.genfromtxt('../sample_data.csv', delimiter=",")
df = np.delete(df, 0, 1)[:, 0:5]

In [24]:
#generating a sample dataset

n = 6
p = 2
df = np.zeros((p*n, 12))
for i in range(1, p+1):
    df[(i-1)*n:i*n, 0] = np.ones(n) + i - 1
    df[(i-1)*n:i*n, 1] = np.ones(n) + i - 1
    df[(i-1)*n:i*n, 2] = np.linspace(0, 10, n) + 2 + i
    df[(i-1)*n:i*n, 3] = np.linspace(0, 10, n) + i
    df[(i-1)*n:i*n, 4] = np.linspace(0, 10, n)
    df[(i-1)*n:i*n, 5] = np.zeros(n)
    df[(i-1)*n:i*n, 6:12] = np.zeros((n, 6))

In [25]:
np.savetxt("../Traj_test_data_1.tif.csv", df, delimiter=",")

In [26]:
folder = '../'
name = 'test_data'
cut = 1
totvids = 1
conversion = (1, 1, 1)

In [31]:
def MSD_iteration(folder, name, cut=1, totvids=1, conversion=(1, 1, 1)):
    """
    Cleans up data for MSD analysis from csv files.  Outputs in form of
    dictionaries.
    """
    frames = 0
    trajectory = dict()
    tots = dict()  # Total particles in each video
    newtots = dict()  # Cumulative total particles.
    newtots[0] = 0
    tlen = dict()
    tlength = dict()
    tlength[0] = 0

    for num in range(1, totvids + 1):
        trajectory[num] = np.genfromtxt(folder+'Traj_{}_{}.tif.csv'.format(name, num), delimiter=",")
        trajectory[num] = np.delete(trajectory[num], 0, 1)

        tots[num] = trajectory[num][-1, 0].astype(np.int64)
        newtots[num] = newtots[num-1] + tots[num]

        tlen[num] = trajectory[num].shape[0]
        tlength[num] = tlength[num-1] + tlen[num]

        if np.max(trajectory[num][:, 1]) > frames:
            frames = int(np.max(trajectory[num][:, 1]))

    placeholder = np.zeros((tlength[totvids], 11))

    for num in range(1, totvids + 1):
        placeholder[tlength[num-1]:tlength[num], :] = trajectory[num]
        placeholder[tlength[num-1]:tlength[num], 0] = placeholder[tlength[num-1]:tlength[num], 0] + newtots[num-1]

    dataset = dict()
    rawdataset = np.zeros(placeholder.shape)
    particles = placeholder[:, 0]
    total = int(max(particles))
    total1 = total + 1
    rawdataset = placeholder[:, :]

    fixed = np.zeros(placeholder.shape)
    fixed[:, 0:2] = rawdataset[:, 0:2]
    fixed[:, 2:4] = conversion[0] * rawdataset[:, 2:4]
    fixed[:, 4] = conversion[2] * rawdataset[:, 4]

    x = np.zeros((frames+1, total1))
    y = np.zeros((frames+1, total1))
    xs = np.zeros((frames+1, total1))
    ys = np.zeros((frames+1, total1))

    nones = 0
    cutoff = cut
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)

        if max1 - min1 < cutoff:
            nones = nones + 1
        else:
            holdplease = fillin2(fixed[min1:max1+1, 0:5])
            x[int(holdplease[0, 1]):int(holdplease[-1, 1])+1, num - nones - 1] = holdplease[:, 2]
            y[int(holdplease[0, 1]):int(holdplease[-1, 1])+1, num - nones - 1] = holdplease[:, 3]

            xs[0:int(holdplease[-1, 1])+1-int(holdplease[0, 1]), num - nones - 1] = holdplease[:, 2]
            ys[0:int(holdplease[-1, 1])+1-int(holdplease[0, 1]), num - nones - 1] = holdplease[:, 3]

    total1 = total1 - nones - 1
    x_m = x[:, :total1]
    y_m = y[:, :total1]
    xs_m = xs[:, :total1]
    ys_m = ys[:, :total1]

    return total1, xs_m, ys_m, x_m, y_m

In [35]:
MSD_iteration(folder, name)

(2, array([[  1.,   2.],
        [  1.,   2.],
        [  3.,   4.],
        [  3.,   4.],
        [  5.,   6.],
        [  5.,   6.],
        [  7.,   8.],
        [  7.,   8.],
        [  9.,  10.],
        [  9.,  10.],
        [ 11.,  12.],
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.]]), array([[  0.,   0.],
        [  0.,   0.],
        [  2.,   2.],
        [  2.,   2.],
        [  4.,   4.],
        [  4.,   4.],
        [  6.,   6.],
        [  6.,   6.],
        [  8.,   8.],
        [  8.,   8.],
        [ 10.,  10.],
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.]]), array([[  0.,   0.],
        [  0.,   0.],
        [  0.,   0.],
        [  1.,   0.],
        [  1.,   2.],
        [  3.,   2.],
        [  3.,   4.],
        [  5.,   4.],
        [  5.,   6.],
        [  7.,   6.],
        [  7.,   8.],
        [  9.,   8.],
        [  9.,  10.],
        [ 11.,  10.],
        [  0.,  12.]]), arr

In [30]:
type(total1)

int

In [34]:
x_m.shape

(15, 2)

In [36]:
np.any?